# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core
from azureml.core import Workspace, Experiment, ScriptRunConfig
from azureml.core.dataset import Dataset
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal, RandomParameterSampling, choice, BanditPolicy
from azureml.widgets import RunDetails
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import os
import pandas as pd
import requests

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'exp_iris_hyperdrive'

experiment=Experiment(ws, experiment_name)

In [3]:
iris = load_iris()

# Combine features and target into a single DataFrame
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

# Register the dataset
datastore = ws.get_default_datastore()
iris_dataset = Dataset.Tabular.register_pandas_dataframe(
    name='iris_dataset',
    dataframe=iris_df,
    target=(datastore, 'iris_dataset')
)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

* Early Termination Policy: The BanditPolicy terminates under-performing runs early to save resources, based on a 10% slack factor, evaluating every iteration after an initial delay of 5 iterations.
* Hyperparameter: RandomParameterSampling is used to explore a set of values for the number of estimators and max depth of the model, enhancing the chances of finding optimal hyperparameters.
* Hyperdrive Configuration: The HyperDriveConfig ensures efficient use of resources by running up to 4 trials concurrently out of a maximum of 20, optimizing for accuracy with the selected hyperparameters and early stopping criteria.

In [4]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--n_estimators": choice(50, 100, 150, 200),
    "--max_depth": choice(5, 10, 15, 20)
})

#TODO: Create your estimator and hyperdrive config
env = Environment.from_conda_specification(name='sklearn_env', file_path='environment.yml')
script_config = ScriptRunConfig(source_directory=os.getcwd(), 
                                script='train.py', 
                                compute_target='notebook260384',
                                environment=env)

hyperdrive_run_config = HyperDriveConfig(run_config=script_config,
                                         hyperparameter_sampling=param_sampling,
                                         policy=early_termination_policy,
                                         primary_metric_name="Accuracy",
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=20,
                                         max_concurrent_runs=4)

In [5]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [12]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n n_estimators:', parameter_values[3])
print('\n max_depth:', parameter_values[1])

Best Run Id:  HD_0901dd13-5975-46cb-aafe-eb9cd49d41d2_13

 Accuracy: 1.0

 n_estimators: 200

 max_depth: 5


In [8]:
#TODO: Save the best model
model = best_run.register_model(
    model_path='outputs/hyperdrive_model.pkl',
    model_name='hyperdrive_best_model',
    tags={'Method':'Hyperdrive'},
    properties={'Accuracy': best_run_metrics['Accuracy']}
)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [9]:
env = Environment.from_conda_specification(name='sklearn_env', file_path='environment.yml')
inference_config = InferenceConfig(entry_script='score_hyperdrive.py', environment=env)

# Deploy the model as a web service
service_name = 'service-hyperdrive'
inference_config = InferenceConfig(entry_script="score_hyperdrive.py", environment=env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-06-08 17:16:03+00:00 Creating Container Registry if not exists.
2024-06-08 17:16:03+00:00 Registering the environment.
2024-06-08 17:16:04+00:00 Use the existing image.
2024-06-08 17:16:04+00:00 Generating deployment configuration.
2024-06-08 17:16:05+00:00 Submitting deployment to compute.
2024-06-08 17:16:12+00:00 Checking the status of deployment service-hyperdrive..
2024-06-08 17:18:07+00:00 Checking the status of inference endpoint service-hyperdrive.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [17]:
scoring_uri = service.scoring_uri
data = {"data": [iris.data[10].tolist()]}
headers = {'Content-Type': 'application/json'}

response = requests.post(scoring_uri, json=data, headers=headers)
print("Prediction:", response.json())

Prediction: {"result": [0]}


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

# Delete the web service
service.delete()

# Shutdown all the computes that have been used
for compute_name in ws.compute_targets:
    ws.compute_targets[compute_name].stop()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

